# Data Analysis

In this notebook, we analyse the (synthetic) data from our ficticious bank account. It is provided in CSV format and our goal is to create a graphical overview over our monthly expenses by category. 

In [ ]:
import pandas as pd
import matplotlib.pylab as plt
plt.style.use("ggplot")
from IPython.display import display, HTML
import numpy as np
import hashlib
from collections import OrderedDict, defaultdict
import sqlite3 as sql
import os
from glob import glob
from datetime import datetime
import sys
from calendar import monthrange


### Exploring unknown data

Every bank provides account statements in a different format. Panda's read_csv function can try to find certain properties of how the data is saved on its own, one example being the separator, but we will have to find the structure ourselves. So, as a first step, it makes sense to simply plot the first few rows and see what they contain.

In [ ]:
data = pd.read_csv("money_bin_201511_2015121.csv")
display(data.loc[range(10), :])

Next, we have to deal with the datatype of the columns. The date column should ideally contain datetime objects and the amount column should be typed as float. Let's see if that is true and adjust the types if needed:

In [ ]:
data.fillna("", inplace=True)

for i in list(data.columns):
    print("Column {0} \t\t is of type {1}".format(i, data[i].dtype))
    
data["Date"] = pd.to_datetime(data["Date"])
data["ID"] = data["ID"].apply(lambda x: int(x) if x != "" else 0)

print("--------------------")
for i in list(data.columns):
    print("Column {0} is of type {1}".format(i, data[i].dtype))

### Preparing the data

For our analysis, we do not need to column 'ID'. Therefore, we have the possibility of merging it with the subject or dropping it altogether. In this example, we will do both, as we first merge the ID column into subject and then drop it. In a final step, the columns are reordered. 

In [ ]:
data["Subject"] = data[["Subject", "ID"]].apply(lambda x: " ".join([str(y) for y in x]), axis=1)
data.drop(["ID"], 1, inplace=True)
data = data[["Date", "Recipient", "Subject", "Amount"]]

In [ ]:
data = data[["Date", "Recipient", "Subject", "Amount"]]
display(data)

### Data wrangling

In a real world setting, we are not going to have only one csv file but typically many files that ideally do not overlap but cannot be guaranteed to do so. We therefore assign a unique identifier to each entry. A simple way to do that is to join all entries into a single string and calculate its md5 checksum. The problem with this approach is that when a payment is made twice on the same day, say two cups of coffee at different times, this approach will fail. Therefore, we include the relative index of payment which makes the hashsum unique. 

In [ ]:
insert_frame = pd.DataFrame(     
    columns=["Date", "Recipient", "Subject", "Amount", "Category", "SimpleChecksum", "Checksum"],
    index=np.arange(data.shape[0]))

idx = 0
for d in pd.date_range(data["Date"].min(), data["Date"].max(), freq="1d"):
    relevant_data = data.loc[data["Date"] == d, :]

    for rd_idx, rd in enumerate(relevant_data.iterrows()):
        checksum = hashlib.md5(str(rd_idx).join([str(s) for s in rd[1]]).encode()).hexdigest()                
        simple_checksum = hashlib.md5("".join([str(s) for s in rd[1]]).encode()).hexdigest()                
        entry = [str(rd[1]["Date"]), rd[1]["Recipient"], rd[1]["Subject"], float(rd[1]["Amount"]), "", simple_checksum, checksum]
        insert_frame.loc[idx, :] = entry
        idx += 1

### Processing data

The last step in processing the new dataset is to assign categories to those payments that we have encountered previously. In this synthetic example, we know the categories quite well, but in a real world dataset, there might be some entries that need manual categorization at least once. Examples include cash expenses, new restaurants and new stores.

On a technical level, we are simply going to check if a certain string is present in either the recipient or the subject or if a combination is true. We are going to save this in an OrderedDict. This has the advantage of being able to set up some broad rules like anything containing the string "Mart" is categorized as "Grocery Store" that can be overriden with a more specific rule saying for example that "Martin's Burger Place" is categorized as "Restaurant". An ordinary python dict in versions <3.6 do not care about the order, which may lead to wrong classification.

In [ ]:
expns = OrderedDict()

expns["CoffeeBrothers"] = ("Coffee", "CoffeeBrothers", "Expenses::Restaurant")
expns["BeansBeansBeans"] = ("Coffee", "BeansBeansBeans", "Expenses::Restaurant")
expns["Who likes dough?"] = ("Bakery", "Who likes dough?", "Expenses::Restaurant")
expns["BestBakers"] = ("Bakery", "BestBakers", "Expenses::Restaurant")
expns["Cuisine Francaise"] = ("Restaurant", "Cuisine Francaise", "Expenses::Restaurant")
expns["Best Burgers"] = ("Restaurant", "Best Burgers", "Expenses::Restaurant")
expns["Gimme Gyros"] = ("Restaurant", "Gimme Gyros", "Expenses::Restaurant")
expns["Bernhard Bratwurst"] = ("Restaurant", "Bernhard Bratwurst", "Expenses::Restaurant")
expns["Trattoria Accento"] = ("Restaurant", "Trattoria Accento", "Expenses::Restaurant")
expns["Convencience"] = ("Convenience Store", "Convenience", "Expenses::Convenience Stores")
expns["Plumber"] = ("Plumber", "", "Expenses::Living::Repairs")
expns["DIY Furniture"] = ("Furniture", "DIY Furniture", "Expenses::Living::Furniture")
expns["Electrobuddy"] = ("Electronics", "Electrobuddy", "Expenses::Living::Electronics")
expns["Farmer's Market"] = ("Groceries", "Farmer's Market", "Expenses::Living::Groceries")
expns["Franny's Fantastic Food"] = ("Groceries", "Franny's Fantastic Food", "Expenses::Living::Groceries")
expns["Super Foods Market"] = ("Groceries", "Super Foods Market", "Expenses::Living::Groceries")
expns["Gas"] = ("Car", "Gas", "Expenses::Car")
expns["Clothes left and right"] = ("Clothes", "Clothes left and right", "Expenses::Clothes")
expns["Rent"] = ("Rent", "", "Expenses::Living::Rent")
expns["Car Loan"] = ("Car Loan", "", "Expenses::Car")
expns["Insurances"] = ("Insurance", "", "Expenses::Insurance")
expns["Phone"] = ("Phone", "", "Expenses::Communication")

def map_entries(entry, categories):  
    
    # check if a category has been assigned already to avoid overwriting user choices
    try:
        if len(entry["Category"]) > 3: return entry
    except:
        pass
    
    recipient = str(entry["Recipient"]).lower()
    subject = str(entry["Subject"]).lower()
    for k in categories.keys():
        if k.find("|") == -1:
            if k.lower() in recipient or k.lower() in subject:
                if len(categories[k]) == 3:
                    entry["Recipient"], entry["Subject"], entry["Category"] = categories[k]
                elif len(categories[k]) > 3:
                    entry["Category"] = categories[k]
                break
        else:
            r, s = k.lower().split("|")
            if r in recipient and s in subject:
                entry["Recipient"], entry["Subject"], entry["Category"] = categories[k]
                break
    return entry

data = data.apply(lambda x: map_entries(x, expns), 1)

### Putting it all together and dumping to sqlite

Here's how the routine would look in practice. We iterate over all possible csv files and look for overlap before inserting. The results are saved to an sqlite database which makes it easy to search.

In [ ]:
df = pd.DataFrame(     
    columns=["Date", "Recipient", "Subject", "Amount", "Category", "SimpleChecksum", "Checksum"])
sql_file = "money_bin.db"

if os.path.exists(sql_file):
    conn = sql.connect(sql_file)
    curs = conn.cursor()
    sql_data = pd.read_sql_query("SELECT * FROM money_bin", conn)
    df = df.append(sql_data)
    print("Transactions present in database:", len(sql_data))
    curs.close()
    conn.close()

for csv_file in glob("money_bin*.csv"):
    data = pd.read_csv(csv_file)
    data.fillna("", inplace=True)
    data["Date"] = pd.to_datetime(data["Date"])
    data["ID"] = data["ID"].apply(lambda x: int(x) if x != "" else 0)
    data["Subject"] = data[["Subject", "ID"]].apply(lambda x: " ".join([str(y) for y in x]), axis=1)
    data.drop(["ID"], 1, inplace=True)
    data = data[["Date", "Recipient", "Subject", "Amount"]]    

    insert_frame = pd.DataFrame(     
        columns=["Date", "Recipient", "Subject", "Amount", "Category", "SimpleChecksum", "Checksum"],
        index=np.arange(data.shape[0]))
    
    idx = 0
    for d in pd.date_range(data["Date"].min(), data["Date"].max(), freq="1d"):
        relevant_data = data.loc[data["Date"] == d, :]

        for rd_idx, rd in enumerate(relevant_data.iterrows()):
            checksum = hashlib.md5(str(rd_idx).join([str(s) for s in rd[1]]).encode()).hexdigest()                
            simple_checksum = hashlib.md5("".join([str(s) for s in rd[1]]).encode()).hexdigest()                
            entry = [str(rd[1]["Date"]), rd[1]["Recipient"], rd[1]["Subject"], float(rd[1]["Amount"]), "", simple_checksum, checksum]
            insert_frame.loc[idx, :] = entry
            idx += 1

    insert_frame = insert_frame.apply(lambda x: map_entries(x, expns), 1)            
    
    if len(df) != 0:
        duplicates = -insert_frame["Checksum"].isin(df["Checksum"])    
        print("Unique transactions: ", len(insert_frame[duplicates]))
        df = df.append(insert_frame[duplicates])
    else:
        df = df.append(insert_frame)


    print("Updated number of transactions: ", len(df))
    print("-----------------------------------------------------")


conn = sql.connect(sql_file)
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS money_bin (Date TEXT, Recipient TEXT, Subject TEXT, Amount REAL, Category TEXT, SimpleChecksum TEXT, Checksum TEXT)")
cursor.execute("CREATE UNIQUE INDEX IF NOT EXISTS money_bin_idx ON money_bin(Checksum)")
columns = ",".join(["'{0}'".format(c) for c in df.columns.tolist()])
cursor.executemany("INSERT OR IGNORE INTO money_bin ({0}) VALUES (?, ?, ?, ?, ?, ?, ?)".format(columns), df.values.tolist())
conn.commit()
cursor.close()
conn.close()

# Excel interaction

Pandas provides excel read and write support that is just as easy to use as csv export. In combination with *xlwings*, we have a powerful tool at our disposal to analyse and manipulate data in real time. The following routines take care of converting SQL data to Excel and back. If the excel sheet money_bin.xlsx is open, you can manipulate the data in real time.

In [ ]:
import xlwings as xw

def xlsx2sql():
    col_names = ["Date", "Recipient", "Subject", "Amount", "Category", "SimpleChecksum", "Checksum"]
    sql_names =",".join([n + "=?" for n in col_names])
    dfs = [pd.read_excel("money_bin.xlsx", str(y)) for y in [2015, 2016, 2017]]
    xl = pd.concat(dfs).fillna("")
    xl["Date"] = xl["Date"].apply(lambda x: str(pd.to_datetime(x))).values
    xl = xl.apply(lambda x: map_entries(x, expns), 1)

    conn = sql.connect("money_bin.db")
    cursor = conn.cursor()
    for d in xl.iterrows():
        cursor.execute("UPDATE OR IGNORE money_bin SET {0} WHERE Checksum='{1}'".format(sql_names, d[1].values[-1]), d[1].values)
        conn.commit()
    cursor.close()
    conn.close()
    
    
def sql2xlsx():
    conn = sql.connect("money_bin.db")
    columns = ["Date", "Recipient", "Subject", "Amount","Category", "SimpleChecksum", "Checksum"]    
    excel_file = pd.ExcelWriter("money_bin.xlsx", engine="xlsxwriter")

    for year in range(2015, 2018):
        entries = pd.read_sql_query("SELECT * FROM money_bin WHERE Date>='{0}' and Date<='{1}' ORDER BY Date".format(datetime(year, 1, 1), datetime(year, 12, 31)), conn)
        entries = entries.apply(lambda x: map_entries(x, expns), 1)
        entries.to_excel(excel_file, str(year), index=False)

        try:
            wb = xw.Book("money_bin.xlsx".format(account))
            sht = wb.sheets[str(year)]
            sht.range("A2:I16384").value = ""
            sht.range("A2").value = entries.values        
            sht.range("A1").value = columns
        except:
            print("Error setting entries on the fly")
            pass
    conn.close()
    excel_file.save()
    
def show_uncategorized():
    conn = sql.connect("money_bin.db")
    columns = ["Date", "Recipient", "Subject", "Amount", "Category", "SimpleChecksum", "Checksum"]
    for year in range(2015, 2018):
        entries = pd.read_sql_query("SELECT * FROM money_bin WHERE Category='' AND Date>='{0}' and Date<='{1}' ORDER BY Date".format(datetime(year, 1, 1), datetime(year, 12, 31)), conn)
        try:
            wb = xw.Book("money_bin.xlsx")
            sht = wb.sheets[str(year)]
            sht.range("A2:I16384").value = ""
            sht.range("A2").value = entries.values        
            sht.range("A1").value = columns
        except:
            print(sys.exc_info())
            pass  
    conn.close()

Save SQL to Excel

In [ ]:
sql2xlsx()

Write changes back to SQL

In [ ]:
xlsx2sql()

Show only items that are uncategorized in Excel

In [ ]:
show_uncategorized()

# Graphical analysis

### Routines for getting data and categories

In [ ]:
# Get all data for years[0].months[0] to years[1].months[1]
def get_data(years, months):
    try:
        conn = sql.connect("money_bin.db")
        query = "SELECT * FROM money_bin WHERE date>='{0}' and date<='{1}'".format(datetime(years[0], months[0], 1, 0, 0), datetime(years[1], months[1], monthrange(years[1], months[1])[1], 0, 0))
        df = pd.read_sql_query(query, conn)
        conn.commit()
        conn.close()
    except:
        print("Getting data failed")
        print(sys.exc_info())
    return df.dropna()

# Get all data for years[0].months[0] to years[1].months[1]
# by category
def get_data_category(years, months, category):
    try:
        conn = sql.connect("money_bin.db".format(db))
        query = "SELECT * FROM money_bin WHERE Category LIKE '{0}%' AND date>='{1}}' and date<='{2}'".format(category, datetime(years[0], months[0], 1, 0, 0), datetime(years[1], months[1], monthrange(years[1], months[1])[1], 0, 0))
        df = pd.read_sql_query(query, conn)
        df = df.append(sql_entries)
        conn.commit()
        conn.close()
    except:
        print("Getting data by category failed")
        
    return df.dropna()

# Get a list of the n_subcategories most important subcategories
def get_most_relevant_subcategories(category, n_subcategories, years, months, ignore=[]):
    df = get_data(years, months)   
    counts = defaultdict(lambda:float(0))
    for d in df.loc[df.Category.str.startswith(category), ["Category", "Amount"]].iterrows():
        c = d[1]["Category"].replace(category, "").split("::")    
        if len(c) == 1:
            counts[category] -= d[1]["Amount"]
        else:
            if c[1] in ignore:
                continue                        
            counts[c[1]] -= d[1]["Amount"]

    counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    labels = [c[0] for c in counts[0:n_subcategories - 1]]
    return labels

# Return a tuple of account names and balances for the n_subcategories accounts with the largest balance
def get_amounts(category, n_subcategories, years, months, ignore=[]):
    df = get_data(years, months)
    
    counts = defaultdict(lambda:float(0))
    for d in df.loc[df.Category.str.startswith(category), ["Category", "Amount"]].iterrows():
        c = d[1]["Category"].replace(category, "").split("::")    
        if len(c) == 1:
            counts[category] -= d[1]["Amount"]
        else:
            if c[1] in ignore:
                continue                                    
            counts[c[1]] -= d[1]["Amount"]

    counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    labels = [c[0] for c in counts[0:n_subcategories - 1]]
    labels += ["Miscellaneous"]
    amounts = [c[1] for c in counts[0:n_subcategories - 1]]
    amounts += [sum([c[1] for c in counts[n_subcategories - 1:]])]
    return labels, amounts

# Return a tuple of account names and balances for desired subcategories
def get_amounts_by_subcategories(category, subcategories, years, months, ignore=[]):
    df = get_data(years, months)
    
    counts = OrderedDict([(s, 0.) for s in subcategories])
    counts["Miscellaneous"] = 0.
    
    for d in df.loc[df.Category.str.startswith(category), ["Category", "Amount"]].iterrows():
        c = d[1]["Category"].replace(category, "").split("::")        
        if c[1] in subcategories:
            counts[c[1]] -= d[1]["Amount"]
        else:
            if c[1] in ignore:
                continue                        
            counts["Miscellaneous"] -= d[1]["Amount"]

    return list(counts.keys()), list(counts.values())

### Get a list of the most relevant categories

In [ ]:
frst_date = pd.datetime(2015, 1, 1)
last_date = pd.datetime(2017, 7, 31)

categories = get_most_relevant_subcategories("Expenses", 24, [frst_date.year, last_date.year], [frst_date.month, last_date.month])
print(categories)
amounts = OrderedDict([(s, []) for s in categories])
amounts["Miscellaneous"] = []
date_labels = []

for date in pd.date_range(frst_date, last_date, freq="MS"):
    date_labels.append(date.strftime("%B %Y"))
    x, y = get_amounts_by_subcategories("Expenses", categories, [date.year, date.year], [date.month, date.month])
    for cat, am in zip(x, y):
        amounts[cat].append(am)
        

In [ ]:
months = len(pd.date_range(frst_date, last_date, freq="MS"))
fig = plt.figure()
ax = fig.add_subplot(111)

budget_frame = pd.DataFrame()

baseline = np.zeros(months)
for cat, ys in amounts.items(): 
    plt.bar(range(months), ys, bottom=baseline, label=cat)
    baseline += ys
    budget_frame[cat] = ys
    
plt.legend()
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], title='Line', loc='upper left')

box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

try:
    wb = xw.Book("expenses.xlsx")
    sht = wb.sheets["Expenses"]
    sht.range("A2:ZZ16384").value = ""
    sht.range("B2").value = budget_frame.transpose()
    sht.range("C2").value = date_labels
    sht.range("C{0}".format(budget_frame.shape[1] + 4)).value = budget_frame.sum(1).values
    sht.range("A3").expand("down").value = [[v] for v in budget_frame.mean(0).values]
    sht.range("A{0}".format(budget_frame.shape[1] + 4)).value = sum(budget_frame.mean(0).values)
    sht.pictures.add(fig, name="overview", update=True)
except:
    print("Could not add values to sheet")
    print(sys.exc_info())

# Outlook
I hope that you are convinced that handling big datasets in pandas is really easy. A next step would be to take an actual dataset from your bank and see if you can analyse and if the simple keyword search suffices for categorizing transactions. You could also add the functionality of handling multiple accounts and generate more detailed statements based on category and recipient / subject if you want to have a more fine-grained picture of where your money actually flows.